In [1]:
import torch
import torchvision
import numpy as np
import matplotlib.pyplot as plt
import warnings

warnings.filterwarnings("ignore")

In [2]:
# Загружаем датасет из модуля torchvision предназначенного для работы с изображениями
train_dataset = torchvision.datasets.MNIST(
    root='./data',
    train=True,
    transform=torchvision.transforms.ToTensor(),
    download=True
)

test_dataset = torchvision.datasets.MNIST(
    root='./data',
    train=False,
    transform=torchvision.transforms.ToTensor(),
    download=True
)

In [ ]:
train_dataset.data

In [ ]:
print("Train: {}\nTest:  {}".format(train_dataset.data.shape, test_dataset.data.shape))

In [ ]:
train_dataset.data.max()

In [ ]:
train_dataset.targets

In [7]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:
model = torch.nn.Sequential(
    torch.nn.Linear(784, 128),
    torch.nn.ReLU(),
    torch.nn.Linear(128, 10)
)

model.to(device)
# Не нужно применять в финальном слое функцию SoftMax!
# Почему?

In [15]:
# Loss function and GD solver
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters())

In [16]:
# Создадим генератор для данных
batch_size = 512

train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=batch_size,
                                           shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=batch_size,
                                          shuffle=False)

In [ ]:
n_epochs = 10

# Storing the losses for the evaluation
train_losses = np.zeros(n_epochs)
test_losses = np.zeros(n_epochs)

for epoch in range(n_epochs):
    train_loss = []
    test_loss = []

    # Training iteration
    for inputs, targets in train_loader:
        inputs, targets = inputs.to(device), targets.to(device)

        # reshape the input as it must be in NxD shape where N - single observation D - number of features per observation
        inputs = inputs.view(-1, 784)

        # zero parameter gradients
        optimizer.zero_grad()

        # Forward propagation
        outputs = model(inputs)
        loss = criterion(outputs, targets)

        # Backward propagation
        loss.backward()
        optimizer.step()

        # accumulate train losses
        train_loss.append(loss.item())

    # Accumulate train loss and test loss
    train_loss = np.mean(train_loss)

    # Testing iteration
    for inputs, targets in test_loader:
        inputs, targets = inputs.to(device), targets.to(device)
        inputs = inputs.view(-1, 784)
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        test_loss.append(loss.item())
    test_loss = np.mean(test_loss)

    # Save losses
    train_losses[epoch] = train_loss
    test_losses[epoch] = test_loss

    # Console output
    print("Epoch: {}/{}\nTrain Loss: {}    Test Loss:  {}".format(epoch+1, n_epochs, round(train_loss, 4), round(test_loss, 4)))

In [ ]:
# Plot the train loss and test loss per iteration
plt.plot(train_losses, label='train loss')
plt.plot(test_losses, label='test loss')
plt.legend()
plt.show()

In [26]:
n_correct = 0.
n_total = 0.

for inputs, targets in train_loader:
    inputs, targets = inputs.to(device), targets.to(device)
    inputs = inputs.view(-1, 784)
    # Forward propagation
    outputs = model(inputs)

    # Get the predictions
    # torch.max returns max and argmax
    _, predictions = torch.max(outputs, 1) # taking the logits over the column axis
    # update val placeholders for n_correct and n_total
    n_correct += (predictions == targets).sum().item()
    n_total += targets.shape[0]

train_accuracy = n_correct / n_total

n_correct = 0.
n_total = 0.

for inputs, targets in test_loader:
    inputs, targets = inputs.to(device), targets.to(device)
    inputs = inputs.view(-1, 784)
    outputs = model(inputs)
    _, predictions = torch.max(outputs, 1)
    n_correct += (predictions == targets).sum().item()
    n_total += targets.shape[0]

test_accuracy = n_correct / n_total

In [ ]:
print("Train Accuracy: {}    Test Accuracy: {}".format(round(train_accuracy, 4), round(test_accuracy, 4)))

In [29]:
# Plot confusion matrix
from sklearn.metrics import confusion_matrix
import numpy as np
import itertools

def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
  """
  This function prints and plots the confusion matrix.
  Normalization can be applied by setting `normalize=True`.
  """
  if normalize:
      cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
      print("Normalized confusion matrix")
  else:
      print('Confusion matrix, without normalization')

  print(cm)

  plt.imshow(cm, interpolation='nearest', cmap=cmap)
  plt.title(title)
  plt.colorbar()
  tick_marks = np.arange(len(classes))
  plt.xticks(tick_marks, classes, rotation=45)
  plt.yticks(tick_marks, classes)

  fmt = '.2f' if normalize else 'd'
  thresh = cm.max() / 2.
  for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
      plt.text(j, i, format(cm[i, j], fmt),
               horizontalalignment="center",
               color="white" if cm[i, j] > thresh else "black")

  plt.tight_layout()
  plt.ylabel('True label')
  plt.xlabel('Predicted label')
  plt.show()

In [ ]:
# get all predictions in an array and plot confusion matrix

x_test = test_dataset.data.numpy()
y_test = test_dataset.targets.numpy()
p_test = np.array([])
for inputs, targets in test_loader:
  # move data to GPU
  inputs = inputs.to(device)

  # reshape the input
  inputs = inputs.view(-1, 784)

  # Forward pass
  outputs = model(inputs)

  # Get prediction
  _, predictions = torch.max(outputs, 1)

  # update p_test
  p_test = np.concatenate((p_test, predictions.cpu().numpy()))

cm = confusion_matrix(y_test, p_test)
plot_confusion_matrix(cm, list(range(10)))

In [ ]:
# Show some misclassified examples
misclassified_idx = np.where(p_test != y_test)[0]
i = np.random.choice(misclassified_idx)
plt.imshow(x_test[i], cmap='gray')
plt.title("True label: %s Predicted: %s" % (y_test[i], int(p_test[i])));